# Студенческий модуль

Этот набор кода выполнит за Вас тяжёлую работу по считыванию файлов заданий и расчёту регрессионных моделей, оставив Вам самую лёгкую часть: выбор гиперпараметров и интерпретацию результатов. Запускайте ячейки в том порядке, в котором они записаны.

In [1]:
# Если следующие строки выполняются с ошибками, проверяйте, как Вы ставили sklearn / NumPy / Matplotlib.
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

In [ ]:
# Пожалуйста, положите выданные Вам файлы рядом с этим файлом и укажите ниже их имена:
X_train = np.loadtxt('train_X_00.txt')
y_train = np.loadtxt('train_y_00.txt')
X_test = np.loadtxt('test_X_00.txt')
# Если при выполнении этой ячейки происходит OSError, убедитесь, что файлы лежат в той же папке,
# а также что Вы вводите их имена правильно.

In [ ]:
# Попробуем нарисовать все спектры, чтобы посмотреть, с каким набором данных мы имеем дело
plt.plot(X_train.T); plt.show()

## Метод главных компонент

Выполним анализ главных компонент, разложив данную нам матрицу независимых переменных: $$ \mathbf{X} \approx \mathbf T \mathbf{P}^\top $$

Поскольку главные компоненты полностью аддитивны, нам достаточно начать с большого их количества, а потом отбросить лишние.

При этом будем обращать внимание на график значений сингулярных чисел $\sigma_j$, которые получаются при сингулярном разложении исходной матрицы $ \mathbf X \approx \mathbf U \operatorname{diag}(\sigma_1, \dots, \sigma_n) \mathbf{V}^\top $, а также на долю дисперсии, объясняемой каждым ($j$-ым) компонентом, $||\mathbf{t}_j \mathbf{p}^\top_j||^2 = \sigma_j^2$ от суммарной объяснённой дисперсии:

In [ ]:
# Теоретически, максимальное возможное количество компонентов равно наименьшей
# из размерностей матрицы спектров, но мы начнём с меньшего их количества.
max_ncomp = 16
pc = PCA(n_components = max_ncomp).fit(X_train)
plt.plot(pc.singular_values_)
plt.title('Сингулярные числа')
plt.show()
plt.plot(pc.explained_variance_ratio_)
plt.title('Доля объяснённой дисперсии')
plt.show()

В следующей ячейке рассмотрите график нагрузок $\mathbf{v}_j$, попробуйте различные количества главных компонент и выберите, по-Вашему, наболее подходящее. Полезные, содержащие информацию главные компоненты будут иметь форму спектральных пиков, а лишние - содержать шум:

In [ ]:
# При проведении анализа на главные компоненты я выбираю ... компонент:
# (замените 1 на натуральное число)
PCA_comps = 1
plt.plot(pc.components_[0:PCA_comps,:].T)
plt.show()

## Проекция на латентные структуры

Перейдём к задаче предсказания неизвестных концентраций. Здесь мы будем использовать модель ПЛС-1, которая предсказывает неизвестные компоненты по одному. Можно будет увидеть, как для различных столбцов предсказываемой матрицы $\mathbf Y$ оптимальное количество различается.

Для этого обучим серию ПЛС-моделей с разным количеством компонент и выберем то, где модель не начинает врать при перекрёстной проверке.

In [ ]:
# Рассчитаем среднеквадратическую ошибку при перекрёстной проверке 
# индивидуально для каждого компонента
cv = [
    GridSearchCV(
        PLSRegression(scale = False),
        {'n_components': list(range(1, max_ncomp+1))},
        scoring = 'neg_root_mean_squared_error',
        return_train_score = True
    ).fit(X_train, y_train[:,i])
    for i in range(y_train.shape[1])
]
# Извлечём результаты и сохраним их в виде матриц
ncomps = np.array([
    [p['n_components'] for p in v.cv_results_['params']] for v in cv
]).T
RMSEP = np.array([
    -v.cv_results_['mean_train_score'] for v in cv
]).T
RMSECV = np.array([
    -v.cv_results_['mean_test_score'] for v in cv
]).T

In [ ]:
plt.subplot(1, 2, 1); plt.plot(ncomps, RMSEP);  plt.title('Ошибка предсказания\nна обучающем наборе')
plt.subplot(1, 2, 2); plt.plot(ncomps, RMSECV); plt.title('Ошибка предсказания\nпри перекрёстной проверке')
plt.legend(list(range(1, y_train.shape[1]+1))); plt.show()

In [ ]:
# Для столбцов [1, 2, 3, 4, 5] я выбираю ... компонент:
# Замените 1 на число компонент (натуральное).
n_comps = (1, 1, 1, 1, 1)

Обучим модели с выбранными количествами компонент, а также нарисуем графики весов PLS (матрицы $\mathbf{W}$):

In [ ]:
models = []
for i in range(len(n_comps)):
    m = PLSRegression(
        n_components = n_comps[i], scale = False
    ).fit(X_train, y_train[:,i])
    models.append(m)
    plt.plot(m.x_weights_)
    plt.title('Веса ПЛС-модели ${\\bf w}_j$\nдля компонента №%d' % (i+1))
    plt.show()
    plt.plot(m.coef_)
    plt.title('Коэффициенты регрессии ${\\bf \\beta}_j$\nдля компонента №%d' % (i+1))
    plt.show()
    
predictions = []
for i in range(len(n_comps)):
    predictions.append(models[i].predict(X_test)[:,0])

np.savetxt('y_test.txt', np.array(predictions).T)

Если Вы дошли до этого момента, у Вас должен был получиться файл с предсказанными концентрациями, `y_test.txt`. Отправьте его на <mailto:ikrylov@laser.chem.msu.ru>. Туда же пришлите графики матриц весов и объяснения, почему Вы считаете, что данные количества компонент являются оптимальными.

Если что-то не получается, пожалуйста, обращайтесь на тот же адрес, я отвечу.